In [4]:
import torch
import transformers

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [15]:
sentence = "Preposterous, prepare I'm flabbergasted!"
input_ids = tokenizer(sentence, return_tensors='pt').input_ids
input_ids

tensor([[37534,  6197,   516,    11,  8335,   314,  1101,   781,   397,  3900,
          8992,     0]])

In [18]:
# words --> tokens --> Unique ID --> vector embed
tokenizer.decode(37534), tokenizer.decode(6197), tokenizer.decode(8335) 

('Prep', 'oster', ' prepare')

In [20]:
for token_id in input_ids[0]:
    print(tokenizer.decode(token_id))

Prep
oster
ous
,
 prepare
 I
'm
 fl
ab
berg
asted
!


In [51]:
sentence = "I skipped across the"
input_ids = tokenizer(sentence, return_tensors='pt').input_ids

In [52]:
for token_id in input_ids[0]:
    print(tokenizer.decode(token_id))

I
 skipped
 across
 the


In [53]:
gpt2 = AutoModelForCausalLM.from_pretrained('gpt2')

In [54]:
outputs = gpt2(input_ids)

In [55]:
print(outputs.logits.shape)
outputs.logits

torch.Size([1, 4, 50257])


tensor([[[ -39.3084,  -39.0100,  -41.8374,  ...,  -46.9337,  -44.9074,
           -39.5149],
         [ -95.4915,  -95.7003, -100.9737,  ..., -105.3830, -105.3877,
           -97.9453],
         [ -77.7412,  -78.2803,  -81.9967,  ...,  -85.0172,  -86.6475,
           -80.5391],
         [ -89.3046,  -87.9105,  -89.7545,  ...,  -90.2860,  -92.4808,
           -88.3736]]], grad_fn=<UnsafeViewBackward0>)

In [56]:
final_logits = gpt2(input_ids).logits[0,-1]
final_logits

tensor([-89.3046, -87.9105, -89.7545,  ..., -90.2860, -92.4808, -88.3736],
       grad_fn=<SelectBackward0>)

In [57]:
final_logits.argmax() # Token ID <--> Index location logits

tensor(4675)

In [58]:
tokenizer.decode(final_logits.argmax())

' street'

In [59]:
top10_logits = torch.topk(final_logits, 10)

In [60]:
for index in top10_logits.indices:
    print(tokenizer.decode(index))

 street
 line
 room
 country
 river
 border
 pond
 bridge
 floor
 road


In [61]:
final_logits.softmax(dim=0)

tensor([6.1705e-07, 2.4875e-06, 3.9347e-07,  ..., 2.3124e-07, 2.5757e-08,
        1.5653e-06], grad_fn=<SoftmaxBackward0>)

In [63]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)

In [66]:
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index)} -- {value.item():.1%}")

 street -- 3.8%
 line -- 3.2%
 room -- 2.1%
 country -- 1.8%
 river -- 1.6%
 border -- 1.5%
 pond -- 1.5%
 bridge -- 1.3%
 floor -- 1.3%
 road -- 1.2%
